In [8]:
import pandas as pd
import os

In [15]:
AA_serials = [item for item in os.listdir('../') if 'AA' in item]
AA_serial = 'AA-202310-001'

# Define path of an Aquagar machine
AA_machine_folder_path = f'../{AA_serial}'

# Define path of Raw Data folders
AA_RawData_folder_path = os.path.join(AA_machine_folder_path, 'Raw Data')
experiment_folders_paths = [os.path.join(AA_RawData_folder_path, item) for item in os.listdir(AA_RawData_folder_path)[1:]]

In [19]:
human_checked_results_df_list = []
for experiment_folder_path in experiment_folders_paths:
    # Id
    plate_results_excel_path = os.path.join(experiment_folder_path, 'outputs', 'results.xlsx')
    if os.path.exists(plate_results_excel_path):
        excel_options = {'dtype': {'Fecha': str, 'Muestra': str, 'Plate_ID':str},  # Specify data types for columns 
                         }
        results_df = pd.read_excel(plate_results_excel_path, **excel_options)
        if (results_df['HumanChecked']==0).all(): 
            human_checked_results_df_list.append(results_df)
   

In [20]:
# Drop 'HumanChecked' column and sort acendingly according to date
updated_df = pd.concat(human_checked_results_df_list, axis = 0).drop(columns='HumanChecked').sort_values(by='FechaExperimento', ascending=True) 

In [21]:
updated_df

,FechaExperimento,Muestra,Plate_ID,Vibrio,Aeromonas,Photobacterium,Staphyloccocus,Micrococcus,Microbiota
0,2024-01-05,Nombre muestra fila de arriba,0021,🟢,🟢,🟢,🟢,🟢,NaN
1,2024-01-05,Nombre muestra fila de abajo,0021,🟢,🟢,🟢,🟢,🟢,NaN


In [22]:
leyenda_df = pd.DataFrame({'Presencia Patogénica': ['🟢 Presencia nula',
              '🟡 Presencia baja',
              '🟠 Presencia media', 
              '🔴 Presencia alta']})

In [24]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Alignment, PatternFill, Border, Side

df = updated_df

# Destination file
excel_path = os.path.join(AA_machine_folder_path, AA_serial + " Resultados.xlsx")

# Create a Pandas Excel writer using openpyxl
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    # Write the DataFrame to the Excel file
    updated_df.to_excel(writer, sheet_name='Resultados', index = False)
    leyenda_df.to_excel(writer, sheet_name='Leyenda', index = False)

    # Access the Excel writer's workbook and sheet
    workbook = writer.book
    sheet = writer.sheets['Resultados']

    # Define cell formatting styles
    centered_alignment = Alignment(horizontal='center', vertical='center')
    lighter_gray_fill = PatternFill(start_color='F0F0F0', end_color='F0F0F0', fill_type='solid')
    no_border = Border()

    # Apply formatting to all cells in the sheet
    for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
        for cell in row:
            cell.alignment = centered_alignment
            cell.fill = lighter_gray_fill
            cell.border = no_border
            cell.number_format = '@'

    # Save the workbook to the Excel file
    workbook.save(excel_path)